Ok, all set up and we can jump into coding. First and foremost, let’s import some necessary libraries: pandas.io.json(to tranforming json file into a pandas dataframe library), folium (plotting library), geopy.geocoders (module to convert an address into latitude and longitude values), and requests (library to handle requests).

In [10]:
#reading our dataset
import pandas as pd 
neighborhoods_subset = pd.read_json(r"https://raw.githubusercontent.com/xkjyeah/singapore-postal-codes/master/buildings.json")
neighborhoods_subset.head()

ValueError: Unexpected character found when decoding array value (2)

In [11]:
import json
import pandas as pd
with open(r'https://raw.githubusercontent.com/xkjyeah/singapore-postal-codes/master/buildings.json",', 'r') as f:
    data = json.load(f)
df = pd.DataFrame(data)

OSError: [Errno 22] Invalid argument: 'https://raw.githubusercontent.com/xkjyeah/singapore-postal-codes/master/buildings.json",'

In [ ]:
# Select first building (which in our case is BedokVille):
neighborhood_name = neighborhoods_subset.loc[0, 'Neighborhood']
neighborhood_latitude = neighborhoods_subset.loc[0, 'Latitude'] 
neighborhood_longitude = neighborhoods_subset.loc[0, 'Longitude'] 
# limit of number of venues returned by Foursquare API
LIMIT = 100 
radius = 500

In [ ]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
results = requests.get(url).json()

#https://raw.githubusercontent.com/xkjyeah/singapore-postal-codes/master/buildings.json", lines=True)
#neighborhoods_subset.head()

In [ ]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)

In [ ]:
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

In [ ]:
venues_map = folium.Map(location=[neighborhood_latitude, neighborhood_longitude], zoom_start=15)
# add a red circle marker to represent the BEDOKVILLE
folium.features.CircleMarker(
    [neighborhood_latitude, neighborhood_longitude],
    radius=10,
    color='red',
    popup='BEDOKVILLE',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)
# add all venues as blue circle markers
for lat, lng, label in zip(nearby_venues.lat, nearby_venues.lng, nearby_venues.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)